---
format: 
  typst: default
  html: default
echo: false
jupyter: python3
---

In [ ]:
city = "Corvallis"

In [ ]:
from IPython.display import Markdown
Markdown(f"# {city}")

In [ ]:
import polars as pl
from plotnine import *
from datetime import date
from calendar import month_name, month_abbr
from pyhere import here

In [ ]:
this_month = date(2025, 5, 1)
highlight_color = "#FF5733" # A bright color for highlighting

In [ ]:
tmean_oregon = pl.read_csv(here("data", "tmean-oregon.csv"), schema_overrides={"date": pl.Date})
tmean = tmean_oregon.filter(
    pl.col("city") == city,
)

In [ ]:
#| include: false

tmean.head()

In [ ]:
this = tmean.filter(pl.col("date") == this_month).row(0, named=True)
Markdown(f"{month_name[this['month']]} {this['year']} was {abs(this['tmean_diff']):.1f}°C {this['tmean_direction']} than usual.")

In [ ]:
(
    ggplot(tmean, aes(x="month", y="tmean"))
    + geom_line(aes(group="year"), alpha=0.2)
    + geom_line(aes(y = "tmean_normal"))
    + geom_line(data=tmean.filter(pl.col("year") == 2025), color=highlight_color)
    + geom_point(
        data=tmean.filter(pl.col("date") == this_month), color=highlight_color
    )
    + scale_x_continuous(breaks=list(range(1, 13)), labels=list(month_abbr[1:]))
    + labs(title = f"{city}, OR", x="", y="Mean Temperature (°C)")
    + theme_bw() 
    + theme(figure_size = (8, 4))
)